In [1]:
# @title Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines # Needed for custom legend handles
import seaborn as sns             # Needed for plotting
import os
from joblib import Parallel, delayed # <<< ADD FOR PARALLELISM
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from autogluon.common import space
import json
import numpy as np
import math
from IPython.display import display
import time
import shutil
import gc
import pyarrow
import warnings

# Ignore the specific FutureWarning related to TimeSeriesScorer prediction_length
warnings.filterwarnings(
    "ignore",
    message="Passing `prediction_length` to `TimeSeriesScorer.__call__` is deprecated.*", # Match start of message
    category=FutureWarning,
    module="autogluon.timeseries.metrics.abstract" # Be specific about the source module
)
print("FutureWarning for TimeSeriesScorer prediction_length suppressed.")

# Also ignore the "path already exists" warning from Predictor init
warnings.filterwarnings(
    "ignore",
    message="path already exists!.*", # Match the start of the warning message
    category=UserWarning # This is often a UserWarning, adjust if necessary
)
print("Predictor path already exists warning suppressed.")

FutureWarning for TimeSeriesScorer prediction_length suppressed.
Predictor path already exists warning suppressed.


In [2]:
# @title Base Configuration (User Parameters, Paths)

# ----- Define LOCAL Paths -----
# Use raw strings (r"...") for Windows paths
local_base_path = r"G:\My Drive\code_projects\btc_forecast" # <<< UPDATE TO YOUR LOCAL BASE PATH

# --- Directory Paths ---
datasets_dir = os.path.join(local_base_path, 'datasets')     # Directory containing .feather files
results_base_dir = os.path.join(local_base_path, 'results') # Top-level results directory

# --- Parallelism Control ---
num_concurrent_runs = 7 # <<< SET MAX NUMBER OF PARALLEL ASSET RUNS
print(f"Setting max concurrent runs: {num_concurrent_runs}")

# --- Frequency & Date for Results ---
frequency = '1h' # Explicitly set frequency for filtering files and naming
date_today = pd.Timestamp.now().strftime("%Y%m%d") # Use YYYYMMDD format
results_dir_timedated = os.path.join(results_base_dir, f"{frequency}_{date_today}") # e.g., results/15m_20250506

try:
    os.makedirs(results_dir_timedated, exist_ok=True)
    print(f"Base results directory for this run: {results_dir_timedated}")
except OSError as e:
     print(f"Warning: Could not create results directory {results_dir_timedated}: {e}")

print(f"Looking for datasets in: '{datasets_dir}'")
print(f"Script frequency set to: '{frequency}'")

# --- Column Names ---
timestamp_col = 'date'
potential_target_cols = ['volume', 'open', 'high', 'low', 'close']
all_value_cols = [timestamp_col] + potential_target_cols

# --- User-defined Experiment Parameters ---
prediction_length = 1
max_train_dur = 2 * (60*60) # Reduced for local testing? Adjust as needed.
desired_windows = 10
backtest_strategy = "rolling"
error_metric = "WQL"
desired_quantiles = [0.05, 0.5, 0.95]

# --- Model Selection ---
full_hp = {"AutoETS": {}}
selected_algorithms = ["AutoETS"]
selected_hp = {alg: full_hp[alg] for alg in selected_algorithms if alg in full_hp}
print("\nSelected Models:", selected_algorithms)

# === Experiment Setup ===
candle_counts_to_test = [100, 250, 500, 750, 1000, 1500, 2000]
targets_to_process = potential_target_cols

# ----- Healthspan Evaluation Parameters -----
enable_healthspan_evaluation = True
evaluation_chunk_size = 50
max_evaluation_chunks = 20
healthspan_threshold_std_devs = 1.5

# --- Pandas Display Options ---
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.precision', 0)
pd.set_option('display.max_columns', None)

# --- Store config in a dictionary for passing to parallel function ---
config_params = {
    "datasets_dir": datasets_dir,
    "results_dir_timedated": results_dir_timedated,
    "frequency": frequency,
    "date_today": date_today,
    "timestamp_col": timestamp_col,
    "potential_target_cols": potential_target_cols,
    "all_value_cols": all_value_cols,
    "prediction_length": prediction_length,
    "max_train_dur": max_train_dur,
    "desired_windows": desired_windows,
    "backtest_strategy": backtest_strategy,
    "error_metric": error_metric,
    "desired_quantiles": desired_quantiles,
    "selected_hp": selected_hp,
    "candle_counts_to_test": candle_counts_to_test,
    "targets_to_process": targets_to_process,
    "enable_healthspan_evaluation": enable_healthspan_evaluation,
    "evaluation_chunk_size": evaluation_chunk_size,
    "max_evaluation_chunks": max_evaluation_chunks,
    "healthspan_threshold_std_devs": healthspan_threshold_std_devs,
}

print("\nConfiguration set and stored in config_params.")

Setting max concurrent runs: 7
Base results directory for this run: G:\My Drive\code_projects\btc_forecast\results\1h_20250506
Looking for datasets in: 'G:\My Drive\code_projects\btc_forecast\datasets'
Script frequency set to: '1h'

Selected Models: ['AutoETS']

Configuration set and stored in config_params.


In [3]:
# @title Helper Functions (Serializer & Healthspan Calc)

# --- Define serializer function ---
def default_serializer(obj):
    if isinstance(obj, (np.integer, np.floating, np.bool_)): return obj.item()
    elif isinstance(obj, np.ndarray): return obj.tolist()
    elif isinstance(obj, pd.Timestamp): return obj.isoformat()
    elif isinstance(obj, pd.Timedelta):
         if pd.isna(obj): return None # Handle NaT
         return obj.total_seconds()
    elif hasattr(obj, 'get_hyperparameters') :
        try: return obj.get_hyperparameters()
        except Exception: return str(obj)
    elif isinstance(obj, (dict, list, str, int, float, bool, type(None))): return obj
    else: return str(obj)


def calculate_healthspan(wql_scores, initial_wql, threshold_std_devs, chunk_size):
    """ Calculates healthspan based on when WQL crosses a threshold. """
    if not wql_scores or initial_wql is None or pd.isna(initial_wql):
        # print("  [Healthspan Calc] Cannot calculate: Invalid inputs.") # Less verbose
        return None, None
    valid_scores_list = [float(s) for s in wql_scores if isinstance(s, (int, float, np.number)) and pd.notna(s)]
    if not valid_scores_list:
         # print("  [Healthspan Calc] Cannot calculate: No valid numeric scores found.") # Less verbose
         return None, None
    scores_array = np.array(valid_scores_list)
    wql_std_dev = 0
    if len(scores_array) > 1: wql_std_dev = np.std(scores_array)
    # elif len(scores_array) == 1: print("  [Healthspan Calc] Warning: Only 1 valid score...") # Less verbose

    actual_wql_baseline = -initial_wql
    threshold = actual_wql_baseline + abs(threshold_std_devs * wql_std_dev)
    # print(f"  [Healthspan Calc] Initial Score (-WQL): {initial_wql:.4f}, Actual WQL Baseline: {actual_wql_baseline:.4f}, Std Dev (WQL): {wql_std_dev:.4f}, Threshold (WQL): {threshold:.4f}") # Less verbose

    healthspan_steps = None
    for i, score in enumerate(wql_scores):
        if pd.isna(score): continue
        if float(score) > threshold:
            healthspan_steps = i * chunk_size
            # print(f"  [Healthspan Calc] Threshold crossed at chunk {i+1} (Score: {float(score):.4f}). Estimated healthspan: {healthspan_steps} steps.") # Less verbose
            break
    if healthspan_steps is None and wql_scores:
         # print(f"  [Healthspan Calc] Threshold not crossed within {len(wql_scores)} evaluated chunks.") # Less verbose
         healthspan_steps = len(wql_scores) * chunk_size

    return healthspan_steps, threshold

In [4]:
# @title File Discovery and Main Processing Loop

# --- DEFINE AND CREATE BASE TEMP MODEL DIRECTORY (ACCESSIBLE GLOBALLY WITHIN THIS CELL) ---
local_temp_models_base = "ag_temp_models_local_run" # Define it here
try:
    # Clean up this base temp directory from previous full script runs, if it exists
    if os.path.exists(local_temp_models_base):
        print(f"Removing existing base temporary model directory: {local_temp_models_base}")
        shutil.rmtree(local_temp_models_base)
    os.makedirs(local_temp_models_base, exist_ok=True)
    print(f"Ensured base temporary model directory exists: {local_temp_models_base}")
except OSError as e:
    print(f"Warning: Could not create/clean base temporary model directory {local_temp_models_base}: {e}")
    # If this fails, individual model dirs might still be created in the current working dir
    # depending on how models_dir is constructed in process_asset.
    # For robustness, models_dir in process_asset should ensure it's a subdirectory.

# --- Asset Processing Function (for Parallelism) ---
def process_asset(asset_file_path, config):
    """
    Loads, processes, trains, evaluates, and plots results for a single asset file.
    Args:
        asset_file_path (str): Path to the .feather asset file.
        config (dict): Dictionary containing all configuration parameters.
    Returns:
        list: A list of result dictionaries for all targets/candle counts processed
              for this asset, or an empty list if processing fails early.
    """
    # Extract config parameters (ensure all needed parameters are in the config dict)
    frequency = config["frequency"]
    timestamp_col = config["timestamp_col"]
    potential_target_cols = config["potential_target_cols"]
    all_value_cols = config["all_value_cols"]
    prediction_length = config["prediction_length"]
    max_train_dur = config["max_train_dur"]
    desired_windows = config["desired_windows"]
    backtest_strategy = config["backtest_strategy"]
    error_metric = config["error_metric"]
    desired_quantiles = config["desired_quantiles"]
    selected_hp = config["selected_hp"]
    candle_counts_to_test = config["candle_counts_to_test"]
    targets_to_process = config["targets_to_process"]
    enable_healthspan_evaluation = config["enable_healthspan_evaluation"]
    evaluation_chunk_size = config["evaluation_chunk_size"]
    max_evaluation_chunks = config["max_evaluation_chunks"]
    healthspan_threshold_std_devs = config["healthspan_threshold_std_devs"]
    results_dir_timedated = config["results_dir_timedated"] # Main results dir
    date_today = config["date_today"]
    # local_temp_models_base is now globally accessible in this cell's scope

    asset_filename = os.path.basename(asset_file_path)
    asset_results = []

    print(f"--- Starting Processing: {asset_filename} (PID: {os.getpid()}) ---")

    try:
        asset_name_part = asset_filename.split(f'-{frequency}-futures')[0]
        current_item_id = f"{asset_name_part}_FUT_{frequency}"
        # print(f"[{asset_name_part}] Derived item_id: {current_item_id}") # Less verbose
    except Exception as e:
        print(f"[{asset_filename}] Warning: Could not derive asset name. Skipping. Error: {e}")
        return asset_results

    print(f"[{asset_name_part}] Loading asset data...")
    try:
        df_full_asset = pd.read_feather(asset_file_path)
    except Exception as e:
        print(f"[{asset_name_part}] Error loading asset file: {e}. Skipping.")
        return asset_results

    print(f"[{asset_name_part}] Cleaning and preprocessing...")
    cols_to_keep_asset = [col for col in all_value_cols if col in df_full_asset.columns]
    df_full_asset = df_full_asset[cols_to_keep_asset].copy()
    if timestamp_col not in df_full_asset.columns: print(f"[{asset_name_part}] Error: Timestamp column missing. Skipping."); return asset_results
    asset_targets_available = [col for col in potential_target_cols if col in df_full_asset.columns]
    if not asset_targets_available: print(f"[{asset_name_part}] Error: No target columns found. Skipping."); return asset_results
    df_full_asset[timestamp_col] = pd.to_datetime(df_full_asset[timestamp_col], errors='coerce')
    numeric_cols_asset = asset_targets_available
    for col in numeric_cols_asset: df_full_asset[col] = pd.to_numeric(df_full_asset[col], errors='coerce')
    cols_to_check_na_asset = [timestamp_col] + numeric_cols_asset
    df_full_asset = df_full_asset.dropna(subset=cols_to_check_na_asset).copy()

    if not df_full_asset.empty:
        asset_timestamp_dtype = df_full_asset[timestamp_col].dtype
        is_tz_aware_asset = pd.api.types.is_datetime64_ns_dtype(asset_timestamp_dtype) and df_full_asset[timestamp_col].dt.tz is not None
        if is_tz_aware_asset: current_time_compare = pd.Timestamp.now(tz=df_full_asset[timestamp_col].dt.tz)
        else: current_time_compare = pd.Timestamp.now().tz_localize(None)
        df_full_asset = df_full_asset.sort_values(timestamp_col).reset_index(drop=True)
        df_full_asset = df_full_asset[df_full_asset[timestamp_col] < current_time_compare]

    if df_full_asset.empty: print(f"[{asset_name_part}] DataFrame empty after filtering. Skipping."); return asset_results
    # print(f"[{asset_name_part}] Usable data points: {len(df_full_asset)}") # Less verbose

    print(f"[{asset_name_part}] Starting Target Loop...")
    asset_loop_targets = [t for t in targets_to_process if t in asset_targets_available]

    for current_target_raw in asset_loop_targets:
        target_col = current_target_raw
        print(f"[{asset_name_part}] --> Processing Target: {target_col}")
        df_experiment_base = df_full_asset.copy()
        results_summary_target = []

        valid_candle_counts_for_target = []
        if enable_healthspan_evaluation:
            min_eval_points = evaluation_chunk_size
            valid_candle_counts_for_target = [c for c in candle_counts_to_test if c + min_eval_points <= len(df_experiment_base)]
            if not valid_candle_counts_for_target: print(f"[{asset_name_part}/{target_col}] Skipping: Not enough data for eval."); continue
        else:
            valid_candle_counts_for_target = [c for c in candle_counts_to_test if c >= prediction_length + 1]
            if not valid_candle_counts_for_target: print(f"[{asset_name_part}/{target_col}] Skipping: Not enough data for train."); continue

        print(f"[{asset_name_part}/{target_col}] Testing candle counts: {valid_candle_counts_for_target}")
        for candle_count in valid_candle_counts_for_target:
            # print(f"[{asset_name_part}/{target_col}] ---> Running Candle Count: {candle_count}") # Less verbose
            iteration_start_time = time.time()
            fit_successful = False; best_score_val_internal = None; error_message = None; fit_num_val_windows = 0; train_data = None
            predictor = None; external_eval_scores = []; healthspan_steps = None; wql_threshold = None
            num_chunks_to_run = 0; training_data_end_time = None

            # Use local_temp_models_base defined outside this function
            path_suffix = f"{asset_name_part}_{frequency}_{target_col}_{backtest_strategy}_{candle_count}cndl"
            models_dir = os.path.join(local_temp_models_base, path_suffix)

            try: # Create specific temp model dir for this run
                os.makedirs(models_dir, exist_ok=True) # It's okay if it exists due to some race, AG warns
            except OSError as e:
                print(f"[{asset_name_part}/{target_col}/{candle_count}] Error creating model sub-directory {models_dir}: {e}. Skipping.")
                results_summary_target.append({'target': target_col,'asset': asset_name_part,'candle_count': candle_count,'best_score_val_internal': None,'error': f'Local model dir error: {e}','time_taken_seconds': time.time() - iteration_start_time,'num_data_points_train': 0,'num_val_windows_used': 0,'external_eval_wql_scores': [],'healthspan_steps': None,'wql_threshold': None})
                continue

            # Data Slicing logic (kept as is, was working)
            df_train_filtered = pd.DataFrame(); df_eval_slice = pd.DataFrame(); num_eval_points_needed = 0
            if enable_healthspan_evaluation:
                max_possible_eval_points = max(0, len(df_experiment_base) - candle_count)
                max_possible_chunks = max_possible_eval_points // evaluation_chunk_size
                num_chunks_to_run = min(max_evaluation_chunks, max_possible_chunks)
                num_eval_points_needed = num_chunks_to_run * evaluation_chunk_size
                if num_chunks_to_run > 0:
                    df_eval_slice = df_experiment_base.tail(num_eval_points_needed).copy()
                    train_end_index = len(df_experiment_base) - num_eval_points_needed
                    train_start_index = max(0, train_end_index - candle_count)
                    df_train_filtered = df_experiment_base.iloc[train_start_index:train_end_index].copy()
                else: df_train_filtered = df_experiment_base.tail(candle_count).copy()
            else: df_train_filtered = df_experiment_base.tail(candle_count).copy()

            # --- Prepare Training Data (and rest of the logic from your loop) ---
            # ... (The entire rest of your candle_count loop logic goes here) ...
            # ... (Make sure 'current_item_id' is used for df_train_filtered["item_id"]) ...
            # ... (All print statements inside here should ideally include asset_name_part/target_col/candle_count for clarity) ...

            # --- Start of embedded logic from previous candle_count loop ---
            num_points_in_train = len(df_train_filtered)
            if df_train_filtered.empty or num_points_in_train < prediction_length + 1:
                 error_msg_detail = 'Empty training slice' if df_train_filtered.empty else f'Insufficient training data ({num_points_in_train})'
                 print(f"[{asset_name_part}/{target_col}/{candle_count}] Error: {error_msg_detail}. Skipping.")
                 results_summary_target.append({'target': target_col,'asset': asset_name_part,'candle_count': candle_count,'error': error_msg_detail,'best_score_val_internal': None,'time_taken_seconds': time.time()-iteration_start_time,'num_data_points_train':0,'num_val_windows_used': 0,'external_eval_wql_scores': [],'healthspan_steps': None,'wql_threshold': None})
                 if os.path.exists(models_dir): shutil.rmtree(models_dir, ignore_errors=True); continue

            df_train_filtered["item_id"] = current_item_id
            id_column_name = "item_id"

            if df_train_filtered[timestamp_col].dt.tz is not None:
                try: df_train_filtered[timestamp_col] = df_train_filtered[timestamp_col].dt.tz_localize(None)
                except Exception as tz_e: print(f"TZ Error: {tz_e}"); results_summary_target.append({'target': target_col,'asset': asset_name_part, 'candle_count': candle_count, 'error': f'TZ conversion failed: {tz_e}', 'best_score_val_internal': None, 'time_taken_seconds': time.time()-iteration_start_time, 'num_data_points_train':num_points_in_train, 'num_val_windows_used': 0, 'external_eval_wql_scores': [], 'healthspan_steps': None, 'wql_threshold': None}); if os.path.exists(models_dir): shutil.rmtree(models_dir, ignore_errors=True); continue

            training_data_end_time = df_train_filtered[timestamp_col].iloc[-1]

            try:
                required_cols_tsdf = [id_column_name, timestamp_col, target_col]
                cols_for_tsdf = [col for col in df_train_filtered.columns if col in required_cols_tsdf or col in potential_target_cols]
                train_data = TimeSeriesDataFrame.from_data_frame(df_train_filtered[cols_for_tsdf], id_column=id_column_name, timestamp_column=timestamp_col)
            except Exception as e: print(f"TSDF Error: {e}"); error_message = f"Train TSDF creation: {e}"; results_summary_target.append({'target': target_col,'asset': asset_name_part,'candle_count': candle_count,'best_score_val_internal': None,'error': error_message,'time_taken_seconds': time.time() - iteration_start_time,'num_data_points_train': num_points_in_train,'num_val_windows_used': 0,'external_eval_wql_scores': [],'healthspan_steps': None,'wql_threshold': None}); if os.path.exists(models_dir): shutil.rmtree(models_dir, ignore_errors=True); continue

            fit_num_val_windows = 0; fit_val_step_size = 1
            if desired_windows > 0:
                 total_data_len_val = len(train_data); min_train_for_val = prediction_length; required_for_val = prediction_length * desired_windows * prediction_length
                 if total_data_len_val >= min_train_for_val + required_for_val: fit_num_val_windows = desired_windows; fit_val_step_size = prediction_length
                 else: max_possible = (total_data_len_val - min_train_for_val) // (prediction_length * prediction_length); fit_num_val_windows = max(0, min(desired_windows, max_possible));
                 if fit_num_val_windows > 0: fit_val_step_size = prediction_length

            try: predictor = TimeSeriesPredictor(prediction_length=prediction_length, target=target_col,eval_metric=error_metric, path=models_dir, quantile_levels=desired_quantiles)
            except Exception as e: error_message = f'Predictor init failed: {e}'; print(f"Predictor Init Error: {e}"); results_summary_target.append({'target': target_col,'asset': asset_name_part,'candle_count': candle_count,'best_score_val_internal': None,'error': error_message,'time_taken_seconds': time.time() - iteration_start_time,'num_data_points_train': num_points_in_train,'num_val_windows_used': fit_num_val_windows,'external_eval_wql_scores': [],'healthspan_steps': None,'wql_threshold': None}); if os.path.exists(models_dir): shutil.rmtree(models_dir, ignore_errors=True); continue

            print(f"[{asset_name_part}/{target_col}/{candle_count}] Fitting model...")
            hpo_tune_kwargs = None
            try: predictor.fit(train_data, presets="best_quality", time_limit=max_train_dur, num_val_windows=fit_num_val_windows, val_step_size=fit_val_step_size, refit_full=True, hyperparameters=selected_hp, hyperparameter_tune_kwargs=hpo_tune_kwargs, enable_ensemble=False, verbosity=0); fit_successful = True
            except Exception as fit_e: error_message = f"Fitting failed: {fit_e}"; print(f"ERROR during fitting: {fit_e}")

            best_model_name = "N/A"
            if fit_successful:
                leaderboard_df = None; best_score_val_internal = None
                try:
                    if fit_num_val_windows > 0: leaderboard_df = predictor.leaderboard(silent=True)
                    if leaderboard_df is not None and not leaderboard_df.empty and 'score_val' in leaderboard_df.columns and pd.api.types.is_numeric_dtype(leaderboard_df['score_val']):
                         ets_lb = leaderboard_df[leaderboard_df['model'].str.contains("ETS", na=False)]
                         if not ets_lb.empty: score = ets_lb["score_val"].iloc[0];
                         if pd.notna(score): best_score_val_internal = score; best_model_name = ets_lb["model"].iloc[0]
                except Exception as lb_e: print(f"Leaderboard Error: {lb_e}"); best_score_val_internal = None

            if fit_successful and enable_healthspan_evaluation and not df_eval_slice.empty and num_chunks_to_run > 0:
                external_eval_scores = []
                try:
                    future_data_full = df_eval_slice; future_data_full["item_id"] = current_item_id
                    if future_data_full[timestamp_col].dt.tz is not None: future_data_full[timestamp_col] = future_data_full[timestamp_col].dt.tz_localize(None)
                    eval_cols_for_tsdf = [col for col in future_data_full.columns if col in required_cols_tsdf or col in potential_target_cols]
                    future_tsdf_full = TimeSeriesDataFrame.from_data_frame(future_data_full[eval_cols_for_tsdf], id_column=id_column_name, timestamp_column=timestamp_col)
                    for i in range(num_chunks_to_run):
                        chunk_start_idx = i * evaluation_chunk_size; chunk_end_idx = (i + 1) * evaluation_chunk_size
                        current_chunk_data = future_tsdf_full.iloc[chunk_start_idx:chunk_end_idx]
                        time_diff = pd.NaT
                        try: # Time calculation
                            chunk_index = current_chunk_data.index
                            if not chunk_index.empty:
                                chunk_end_timestamp_obj = chunk_index[-1][1]; chunk_end_time = pd.to_datetime(chunk_end_timestamp_obj)
                                if not isinstance(training_data_end_time, pd.Timestamp): training_data_end_time_ts = pd.to_datetime(training_data_end_time)
                                else: training_data_end_time_ts = training_data_end_time
                                if pd.notna(chunk_end_time) and pd.notna(training_data_end_time_ts): time_diff = chunk_end_time - training_data_end_time_ts
                                else: time_diff = pd.Timedelta(seconds=np.nan)
                            else: time_diff = pd.Timedelta(seconds=np.nan)
                        except Exception: time_diff = pd.Timedelta(seconds=np.nan)

                        if len(current_chunk_data) < prediction_length + 1: external_eval_scores.append({'time_since_train': time_diff, 'WQL': np.nan}); continue
                        try:
                            eval_metrics = predictor.evaluate(current_chunk_data, model='AutoETS_FULL', metrics=[error_metric])
                            wql_score_chunk = eval_metrics.get(error_metric, np.nan)
                            external_eval_scores.append({'time_since_train': time_diff, 'WQL': wql_score_chunk})
                        except Exception as eval_e: print(f"ERROR evaluating chunk {i+1}: {eval_e}"); external_eval_scores.append({'time_since_train': time_diff, 'WQL': np.nan})
                except Exception as ext_eval_setup_e: print(f"Ext Eval Error: {ext_eval_setup_e}"); error_message = f"Ext Eval Error: {ext_eval_setup_e}" if error_message is None else error_message
                if external_eval_scores:
                     wql_values = [item['WQL'] for item in external_eval_scores if pd.notna(item['WQL'])]
                     if wql_values:
                         baseline_wql = -best_score_val_internal if (best_score_val_internal is not None and pd.notna(best_score_val_internal)) else None
                         if baseline_wql is not None: healthspan_steps, wql_threshold = calculate_healthspan(wql_scores=wql_values, initial_wql=baseline_wql, threshold_std_devs=healthspan_threshold_std_devs, chunk_size=evaluation_chunk_size)

            iteration_end_time = time.time()
            results_summary_target.append({'target': target_col,'asset': asset_name_part,'candle_count': candle_count,'best_score_val_internal': best_score_val_internal,'error': error_message,'time_taken_seconds': iteration_end_time - iteration_start_time,'num_data_points_train': num_points_in_train,'num_val_windows_used': fit_num_val_windows,'external_eval_wql_scores': external_eval_scores,'healthspan_steps': healthspan_steps,'wql_threshold': wql_threshold})
            print(f"--- Finished Iteration: Asset={asset_name_part}, Target={target_col}, Candles={candle_count}, Time={results_summary_target[-1]['time_taken_seconds']:.1f}s, Score={best_score_val_internal if best_score_val_internal is not None else 'N/A'}, Healthspan={healthspan_steps if healthspan_steps is not None else 'N/A'} ---")

            if predictor is not None: del predictor; gc.collect()
            if 'train_data' in locals() and train_data is not None: del train_data; gc.collect()
            # Clean up models_dir for this specific iteration if it's temporary *per iteration*
            if os.path.exists(models_dir): shutil.rmtree(models_dir, ignore_errors=True)


        # === END CANDLE COUNT LOOP (for current target/asset) ===
        print(f"\n[{asset_name_part}/{target_col}] Finished Candle Count Loop.")
        # Append results for this target to the main list for this asset
        asset_results.extend(results_summary_target)

        # === Plotting and Saving for current target/asset ===
        if not results_df_target.empty:
            # (The existing plotting and saving logic for a single target/asset combination)
            # Ensure save paths include asset_name_part and results_dir_timedated
            # --- Display Summary Table ---
            results_df_target_display = pd.DataFrame(results_summary_target) # Use the specific list for this target
            display_cols = ['candle_count', 'best_score_val_internal', 'healthspan_steps', 'wql_threshold', 'error', 'time_taken_seconds']
            display_cols_present = [col for col in display_cols if col in results_df_target_display.columns]
            if display_cols_present: display(results_df_target_display[display_cols_present])
            else: display(results_df_target_display)

            # --- Save Detailed JSON Summary FOR THIS TARGET/ASSET ---
            summary_filename = f'{date_today}_{asset_name_part}_{frequency}_{target_col}_{backtest_strategy}_healthspan_summary.json'
            summary_save_path = os.path.join(results_dir_timedated, summary_filename)
            try: # Save JSON (using the serialization helper)
                 with open(summary_save_path, 'w') as f: json.dump(results_summary_target, f, default=default_serializer, indent=4) # Save target specific
                 print(f"[{asset_name_part}/{target_col}] Detailed summary saved: {summary_save_path}")
            except Exception as e: print(f"[{asset_name_part}/{target_col}] Error saving JSON summary: {e}")

            # --- Prepare Data for Enhanced Plot ---
            plot_data_list = []
            for idx, row in results_df_target_display.iterrows(): # Use target-specific df
                cc = row['candle_count']; oos_scores = row.get('external_eval_wql_scores', [])
                if oos_scores:
                    for i_chunk, score_item in enumerate(oos_scores):
                        wql = score_item.get('WQL'); steps = (i_chunk + 0.5) * evaluation_chunk_size
                        plot_data_list.append({'candle_count': cc,'steps_since_train': steps,'WQL': wql,'WQL_neg': -wql if pd.notna(wql) else np.nan})
            plot_df_target_external = pd.DataFrame(plot_data_list) if plot_data_list else pd.DataFrame()
            if not plot_df_target_external.empty: plot_df_target_external.dropna(subset=['steps_since_train', 'WQL_neg'], inplace=True)

            # --- PLOT 1: Internal Score vs Candle Count ---
            results_df_filtered_internal = results_df_target_display.dropna(subset=['best_score_val_internal']).copy()
            if not results_df_filtered_internal.empty:
                # (Plotting logic as before, using asset_name_part and target_col in title/filename)
                plt.figure(figsize=(12, 6)); plt.plot(results_df_filtered_internal['candle_count'], results_df_filtered_internal['best_score_val_internal'], marker='o', linestyle='-')
                plt.title(f'Internal Score (-WQL) vs. Candle Count (Asset: {asset_name_part}, Target: {target_col})'); plt.xlabel('Number of Candles (`candle_count`)'); plt.ylabel(f'Internal Fit Score (-WQL, Higher is Better)')
                plt.grid(True); plt.xticks(rotation=45); plt.tight_layout()
                plot_filename_internal = f'{date_today}_{asset_name_part}_{frequency}_{target_col}_{backtest_strategy}_internal_score_plot.png'
                plot_save_path_internal = os.path.join(results_dir_timedated, plot_filename_internal)
                try: plt.savefig(plot_save_path_internal, dpi=150); print(f"[{asset_name_part}/{target_col}] Internal score plot saved: {plot_save_path_internal}")
                except Exception as e: print(f"[{asset_name_part}/{target_col}] Error saving internal plot: {e}")
                plt.close() # Close plot

            # --- PLOT 2: Enhanced Healthspan Plot ---
            if not plot_df_target_external.empty:
                 print(f"[{asset_name_part}/{target_col}] Generating Enhanced Healthspan Plot...")
                 # (Plotting logic as before, using asset_name_part and target_col in title/filename)
                 plot_dir_fig2 = results_dir_timedated; timestamp_str_fig2 = date_today; script_name_base_fig2 = f"{asset_name_part}_{target_col}"
                 plt.style.use('seaborn-v0_8-whitegrid'); fig, ax = plt.subplots(figsize=(16, 9))
                 candle_counts_plot = sorted(plot_df_target_external['candle_count'].unique())
                 palette = sns.color_palette(n_colors=len(candle_counts_plot)); color_map = dict(zip(candle_counts_plot, palette))
                 sns.lineplot(data=plot_df_target_external, x='steps_since_train', y='WQL_neg', hue='candle_count', hue_order=candle_counts_plot, palette=color_map, marker='.', linewidth=1.5, legend=False, ax=ax)
                 is_legend_handles = [];
                 for cc_plot in candle_counts_plot: # Use cc_plot to avoid conflict
                     is_score_row = results_df_target_display[results_df_target_display['candle_count'] == cc_plot]
                     if not is_score_row.empty: is_score = is_score_row['best_score_val_internal'].iloc[0]
                     if pd.notna(is_score): ax.axhline(y=is_score, color=color_map.get(cc_plot, 'grey'), linestyle=':', linewidth=1.5, alpha=0.9); is_legend_handles.append(mlines.Line2D([], [], color=color_map.get(cc_plot, 'grey'), linestyle=':', linewidth=1.5, label=f'CC {cc_plot} (IS Ref)'))
                 median_wql_neg = plot_df_target_external.groupby('steps_since_train')['WQL_neg'].median(); median_handle = []
                 if not median_wql_neg.empty: ax.plot(median_wql_neg.index, median_wql_neg.values, color='black', linestyle='--', linewidth=2.5, label='Median OOS'); median_handle = [mlines.Line2D([], [], color='black', linestyle='--', linewidth=2.5, label='Median OOS')]
                 threshold_handle = []; avg_actual_wql_threshold = results_df_target_display['wql_threshold'].mean()
                 if pd.notna(avg_actual_wql_threshold): avg_neg_threshold_plot = -avg_actual_wql_threshold; ax.axhline(y=avg_neg_threshold_plot, color='red', linestyle='-.', linewidth=2.0, label=f'Avg Degrad. Threshold (≈ {avg_neg_threshold_plot:.4f})'); threshold_handle = [mlines.Line2D([], [], color='red', linestyle='-.', linewidth=2.0, label=f'Avg Degrad. Threshold (≈ {avg_neg_threshold_plot:.4f})')]
                 oos_legend_handles = [mlines.Line2D([], [], color=color_map.get(cc_plot, 'grey'), marker='.', linewidth=1.5, linestyle='-', label=f'CC {cc_plot} (OOS)') for cc_plot in candle_counts_plot]
                 all_handles = oos_legend_handles + is_legend_handles + median_handle + threshold_handle
                 valid_handles = [h for h in all_handles if h.get_label() and not h.get_label().startswith('_')]; ax.legend(handles=valid_handles, title='Performance (-WQL, Higher is Better)', loc='best', fontsize='small')
                 ax.set_title(f"Model Healthspan: OOS vs IS (Asset: {asset_name_part}, Target: {target_col})", fontsize=16)
                 ax.set_xlabel(f"Steps Since Training (Approx. based on Chunk Size = {evaluation_chunk_size})", fontsize=12); ax.set_ylabel("Performance Score (-WQL, Higher is Better)", fontsize=12)
                 ax.grid(True, which='major', linestyle='--', linewidth='0.5', color='grey'); ax.grid(True, which='minor', linestyle=':', linewidth='0.5', color='lightgrey'); ax.minorticks_on()
                 try:
                     plot_filename = f'{timestamp_str_fig2}_{script_name_base_fig2}_{frequency}_{backtest_strategy}_healthspan_plot_v2.png' # Corrected original variable names used here
                     plot_save_path = os.path.join(plot_dir_fig2, plot_filename)
                     plt.savefig(plot_save_path, bbox_inches='tight', dpi=150)
                     print(f"[{asset_name_part}/{target_col}] Enhanced Healthspan plot saved: {plot_save_path}")
                 except Exception as e: print(f"[{asset_name_part}/{target_col}] Error saving plot: {e}")
                 plt.close(fig)
            else: print(f"\n[{asset_name_part}/{target_col}] Skipping enhanced healthspan plot: No valid external evaluation data.")
        else: print(f"\n[{asset_name_part}/{target_col}] No valid results to plot.")

        # Cleanup for Target
        if 'df_experiment_base' in locals(): del df_experiment_base; gc.collect()
        if 'results_df_target_display' in locals(): del results_df_target_display; gc.collect()
        if 'results_df_filtered_internal' in locals(): del results_df_filtered_internal; gc.collect()
        if 'plot_df_target_external' in locals(): del plot_df_target_external; gc.collect()

    # --- END TARGET LOOP (for current asset) ---
    print(f"\n[{asset_name_part}] Finished Target Loop.")
    # asset_results now contains all results for the current asset (list of lists of dicts)
    # For the overall list, we're extending it directly with results_summary_target,
    # so asset_results list is not strictly needed if appending directly to all_assets_all_targets_results
    # However, if you wanted to return it from the function:
    # return asset_results # Or flatten it before returning

# === END ASSET FILE LOOP ===
print(f"\n\n{'='*20} Finished All Asset Files {'='*20}")

if __name__ == "__main__":
    # Re-fetch files if needed, or assume asset_files_to_process is populated
    if not asset_files_to_process:
        print("No asset files were discovered. Re-run file discovery or check paths.")
    else:
        print(f"\n{'='*20} Starting Parallel Asset Processing ({num_concurrent_runs} jobs) {'='*20}")
        start_parallel_time = time.time()

        results_list_parallel = Parallel(n_jobs=num_concurrent_runs, backend="loky", verbose=10)(
             delayed(process_asset)(file_path, config_params) for file_path in asset_files_to_process
        )

        end_parallel_time = time.time()
        print(f"\n{'='*20} Parallel Asset Processing Finished {'='*20}")
        print(f"Total time for parallel execution: {(end_parallel_time - start_parallel_time)/60:.2f} minutes")

        # --- Flatten results from all parallel runs ---
        print("\nAggregating results from parallel jobs...")
        all_assets_all_targets_results_flat = []
        job_success_count = 0; job_fail_count = 0
        if results_list_parallel: # Check if any results came back
            for single_asset_results_list in results_list_parallel:
                if isinstance(single_asset_results_list, list) and single_asset_results_list: # Job returned a non-empty list
                    all_assets_all_targets_results_flat.extend(single_asset_results_list)
                    job_success_count +=1
                elif isinstance(single_asset_results_list, list) and not single_asset_results_list: # Job returned empty list (e.g. asset skipped early)
                    job_fail_count +=1 # Count as skipped/failed
                else: # Job might have returned None or an exception object if not handled within
                    print(f"Warning: A parallel job did not return a list of results. Result: {type(single_asset_results_list)}")
                    job_fail_count +=1
        print(f"Aggregation complete. Total results records: {len(all_assets_all_targets_results_flat)}")
        print(f"Assets processed (at least one target run): {job_success_count}")
        print(f"Assets potentially failed or skipped entirely: {job_fail_count}")

        # --- Save ALL results combined ---
        if all_assets_all_targets_results_flat:
             all_results_save_path = os.path.join(config_params["results_dir_timedated"], f'{config_params["date_today"]}_{config_params["frequency"]}_ALL_ASSETS_TARGETS_healthspan_summary.json')
             try:
                  # Use the same serialization logic as before
                  serializable_all_results_flat = []
                  for record in all_assets_all_targets_results_flat:
                      new_record = record.copy(); temp_scores = []
                      if 'external_eval_wql_scores' in new_record and new_record['external_eval_wql_scores']:
                          for score_item in new_record['external_eval_wql_scores']:
                               new_score_item = score_item.copy()
                               time_delta = new_score_item.get('time_since_train')
                               if isinstance(time_delta, pd.Timedelta): new_score_item['time_since_train'] = None if pd.isna(time_delta) else time_delta.total_seconds()
                               temp_scores.append(new_score_item)
                          new_record['external_eval_wql_scores'] = temp_scores
                      serializable_all_results_flat.append(new_record)

                  with open(all_results_save_path, 'w') as f: json.dump(serializable_all_results_flat, f, default=default_serializer, indent=4)
                  print(f"\nCOMBINED detailed results summary saved to JSON: {all_results_save_path}")
             except Exception as e:
                  print(f"\nError saving COMBINED results summary: {e}")
        else:
            print("\nNo results generated across all assets to save.")

    # # Final cleanup of the base temporary model directory
    # if os.path.exists(local_temp_models_base):
    #     print(f"\nCleaning up base temporary model directory: {local_temp_models_base}")
    #     try:
    #         shutil.rmtree(local_temp_models_base)
    #         print(f"Successfully removed {local_temp_models_base}")
    #     except OSError as e:
    #         print(f"Error removing {local_temp_models_base}: {e}")
    # else:
    #     print(f"\nBase temporary model directory {local_temp_models_base} not found, no cleanup needed or already cleaned.")


Found 512 asset files to process:
 - 1000APU_USDT_USDT-1h-futures.feather
 - 1000BONK_USDT_USDT-1h-futures.feather
 - 1000BTT_USDT_USDT-1h-futures.feather
 - 1000CAT_USDT_USDT-1h-futures.feather
 - 1000CATS_USDT_USDT-1h-futures.feather
 - 1000FLOKI_USDT_USDT-1h-futures.feather
 - 1000LUNC_USDT_USDT-1h-futures.feather
 - 1000MUMU_USDT_USDT-1h-futures.feather
 - 1000NEIROCTO_USDT_USDT-1h-futures.feather
 - 1000PEPE_USDT_USDT-1h-futures.feather
 - 1000RATS_USDT_USDT-1h-futures.feather
 - 1000TOSHI_USDT_USDT-1h-futures.feather
 - 1000TURBO_USDT_USDT-1h-futures.feather
 - 1000X_USDT_USDT-1h-futures.feather
 - 1000XEC_USDT_USDT-1h-futures.feather
 - 10000COQ_USDT_USDT-1h-futures.feather
 - 10000ELON_USDT_USDT-1h-futures.feather
 - 10000LADYS_USDT_USDT-1h-futures.feather
 - 10000QUBIC_USDT_USDT-1h-futures.feather
 - 10000SATS_USDT_USDT-1h-futures.feather
 - 10000WEN_USDT_USDT-1h-futures.feather
 - 10000WHY_USDT_USDT-1h-futures.feather
 - 1000000BABYDOGE_USDT_USDT-1h-futures.feather
 - 100000

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 tasks      | elapsed:  7.8min
[Parallel(n_jobs=7)]: Done  11 tasks      | elapsed: 15.2min
[Parallel(n_jobs=7)]: Done  18 tasks      | elapsed: 23.3min
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed: 31.6min
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed: 42.3min
[Parallel(n_jobs=7)]: Done  47 tasks      | elapsed: 53.3min
[Parallel(n_jobs=7)]: Done  58 tasks      | elapsed: 67.0min
[Parallel(n_jobs=7)]: Done  71 tasks      | elapsed: 79.5min
[Parallel(n_jobs=7)]: Done  84 tasks      | elapsed: 93.0min
[Parallel(n_jobs=7)]: Done  99 tasks      | elapsed: 108.0min
[Parallel(n_jobs=7)]: Done 114 tasks      | elapsed: 124.0min
[Parallel(n_jobs=7)]: Done 131 tasks      | elapsed: 141.5min
[Parallel(n_jobs=7)]: Done 148 tasks      | elapsed: 158.7min
[Parallel(n_jobs=7)]: Done 167 tasks      | elapsed: 177.1min
[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed: 199.4min
[Par


==================== Parallel Asset Processing Finished ====================
Total time for parallel execution: 551.74 minutes

Aggregating results...
Aggregation complete. Total results records: 17105
Assets processed successfully (at least one target): 509
Assets failed or skipped: 3

COMBINED detailed results summary saved to JSON: G:\My Drive\code_projects\btc_forecast\results\1h_20250506\20250506_1h_ALL_ASSETS_TARGETS_healthspan_summary.json


NameError: name 'local_temp_models_base' is not defined